# Basic Usage 

All the basics of making test cases.

In [1]:
%load_ext nbtest

Tags begin with the `@` symbol. If there's a Python identifier compatible tag in the docstring (e.g. `@answer1`) the cell is added to the cell cache. The cell can be re-run by test code so cells like `@answer1` should not assign variables so they can be modified. 

In [2]:
a = 100 
b = 200 

In [3]:
"""@answer1"""

a < b

True

You can introspect and run cells easily using the API for `nbtest`. 

In [4]:
import nbtest

answer1 = nbtest.find('@answer1')
print("Answer source:", answer1.source)
print("Answer result:", type(answer1.result))
print("Answer result value:", answer1.result.result)

a = 200
b = 100 
print("Re-run answer:", answer1.run())

# run() doesn't affect the cache.
print("Original result value:", answer1.result.result)
 

Answer source: """@answer1"""

a < b

Answer result: <class 'IPython.core.interactiveshell.ExecutionResult'>
Answer result value: True


False

Re-run answer: False
Original result value: True


The `%%testing` cell magic provides an important wrapper for test code. Testing cells render feedback for students. Tests run in a separate module namespace, just as they would for normal unit tests. Attributes listed after the testing cell magic are imported into the test namespace so they are available to test code. Imports in testing cells are not shared with the rest of the notebook.

Assertions are a convenient way to do simple checks.

In [5]:
%%testing @answer1

import ast 

# The `answer1` variable contains the cell cache entry 
assert ast.Lt in answer1.tokens, "I don't see less than!"
assert ast.Gt not in answer1.tokens, "I see greater than!"


Test cells always succeed, unless there's a problem with the test code. If there's a problem students are shown your messages.

In [6]:
%%testing @answer1

# Failures don't stop the cell.
assert ast.Lt not in answer1.tokens, "The answer doesn't contain the a less than sign."

Use `shell.push()` to set variables in the notebook namespace. The `run()` function is a convenience for running notebook cells.

In [7]:
%%testing @answer1

# The `shell` variable contains the interactive shell.
shell.push({'a': 1, 'b': 2}) 
assert answer1.run() == True, "1 < 2 != True"

# `run()` optionally pushes variables too.
assert answer1.run({'a': 3, 'b': 2}) == False, "3 < 2 != False"

# Output from running the cell is below 

True

False

Functions that begin with `test` become test cases. Test cases are run and results are rendered below the cell. Test results use the docstring of test functions and assertion errors to inform the student how to fix their problems.

In [8]:
%%testing @answer1

# Functions that start with `test` become a `FunctionTestCase`
def test_lt(): 
    """Testing less than."""
    assert answer1.run({'a': 1, 'b': 2}) == True, "1 < 2 != True"
    assert answer1.run({'a': 3, 'b': 2}) == False, "3 < 2 != False"    

def test_broken_nodoc(): 
    assert answer1.run({'a': 1, 'b': 2}) == False, "1 < 2 != True"

def test_broken_withdoc(): 
    """The test description is used to provide feedback to the learner. They will see this message when the test fails. It's useful to put a hint here."""
    assert answer1.run({'a': 1, 'b': 2}) == False, "1 < 2 != True" 


True

False

True

True

Testing cell source is useful but solutions may contain variables, classes or functions. Those can be tested too.


In [9]:
"""@answer2"""

def add(a, b):
    """My solution function"""
    return a + b 

When solutions contain symbols they must be added to the testing magic to be present in the test namespace.

In [10]:
%%testing add

# Attributes must be listed to be present in the test namespace.
assert add.__doc__ is not None, "add() doesn't have a docstring."

def test_add():
    assert add(1,2) == 3, "1 + 2 != 3"
    assert add(-2,-2) == -4, "-2 + -2 != -4"
    assert add(1,-3) == -2, "1 + -3 != -2"

def test_err():
    """This is a description of the error that has happened."""
    assert add(0,"bogus"), "This is a bogus test."

The test functions are based on Python's `unittest`. Any classes in a testing cell that derive form `unittest.TestCase` are automatically run.

In [11]:
%%testing add 

# Get the full power of TestCase (or write your own)

import unittest

class TestAdd(unittest.TestCase):

    def test_add(self):
        self.assertEqual(add(1,2), 3, "add(1,2)")
        self.assertEqual(add(-2,-2), -4, "add(-2,-2)")
        self.assertEqual(add(1,-3), -2, "add(-1,-3)")

    def test_badd_nodoc(self):
        self.assertEqual(add(1,2), 0, "add(1,2)")

    def test_badd_doc(self):
        """Bad test with documentation."""
        self.assertEqual(add(1,2), 0, "add(1,2)")

    def test_skip_me(self):
        self.skipTest("This test isn't relevant.")

If an attribute is missing the testing cell provides helpful feedback.

In [12]:
%%testing bogus 

# Bad params cause an error. 
pass

The symbol `nbtest_attrs` is a dictionary that contains the attributes given to the `%%testing` cell magic.

In [13]:
%%testing @answer1 @answer2 add

from nbtest import nbtest_attrs

for attr in nbtest_attrs:
    print(f"Attribute: {attr} value: {nbtest_attrs[attr]}")


Attribute: answer1 value: <nbtest.cache.CacheEntry object at 0x7436b0547760>
Attribute: answer2 value: <nbtest.cache.CacheEntry object at 0x7436b0547ac0>
Attribute: add value: <function add at 0x7436b028ee60>
